In [1]:
import os

In [2]:
%pwd

'f:\\DS\\TextSummarizer\\Text-Summarizer\\research'

In [5]:
os.chdir('TextSummarizer/Text-Summarizer/')

In [6]:

%pwd

'f:\\DS\\TextSummarizer\\Text-Summarizer'

In [7]:
import box
from box import ConfigBox

In [17]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_filepath: Path
    unzip_dir: Path

In [9]:
from textSummarizer.constants import *
from textSummarizer.utils.common_utils import read_yaml, create_directories

In [10]:
class ConfigurationManager:

    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def fetch_dataingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_filepath=config.local_data_filepath,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [25]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common_utils import get_size

In [12]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):

        if not os.path.exists(self.config.local_data_filepath):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_filepath
            )
            logger.info(f"{filename} downlaoded with folllowing information: \n{headers}")

        else:
            logger.info(f"file already exists of size: \n{get_size(Path(self.config.local_data_filepath))}")

    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_filepath, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.fetch_dataingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()

except Exception as e:
    raise e

[2024-03-28 09:03:10,647: INFO: common_utils: yaml file: config\config.yaml loaded successfully]
[2024-03-28 09:03:10,651: INFO: common_utils: yaml file: params.yaml loaded successfully]
[2024-03-28 09:03:10,653: INFO: common_utils: Created directory at: artifacts]
[2024-03-28 09:03:10,657: INFO: common_utils: Created directory at: artifacts/data_ingestion]
[2024-03-28 09:03:22,508: INFO: 4214515413: artifacts/data_ingestion/data.zip downlaoded with folllowing information: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 08B2:29EBBE:221257:2BADCA:6604DB98
Accept-Ranges: bytes
Date: Thu, 28 Mar 2024 03:33:12 GMT
Via: 1.1 varnish
X-Served-